
# TDB Matching vs PINT (BIPM2024, GPS→UTC, mk2utc)

Recompute TDB using a standalone path and compare to PINT's `tdbld` for J1909-3744 with:
- Clock chain: mk2utc + gps2utc + BIPM2024 (TT-TAI small part)
- PINT and standalone use the same clock files in `data/clock`
- High-precision MJD parsing and PINT-style normalization (`mjds_to_jds_pulsar`)
- Astropy ephemeris forced to DE440 to match PINT's Einstein/TT↔TDB handling

Outputs:
- Summary statistics (ns)
- Histogram of differences
- Assertion that all TOAs agree within 0.001 ns


In [1]:

import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from bisect import bisect_left
import erfa
from astropy.time import Time
from astropy.coordinates import EarthLocation, solar_system_ephemeris
from pint.models import get_model
from pint.toa import get_TOAs
from pint.observatory import Observatory
from pint.observatory.topo_obs import TopoObs
from pint.pulsar_mjd import mjds_to_jds_pulsar

# Configuration
clock_dir = Path('data/clock').resolve()
par_file = Path('/home/mattm/projects/HSYMT_dump/partim_real/tdb/J1909-3744_tdb.par')
tim_file = Path('/home/mattm/projects/HSYMT_dump/partim_real/tdb/J1909-3744.tim')
mk_loc = EarthLocation.from_geocentric(5109360.133, 2006852.586, -3238948.127, unit='m')
SECS_PER_DAY = np.longdouble(86400.0)

# Use the same ephemeris as PINT (DE440)
solar_system_ephemeris.set('de440')

# Point PINT to local clock dir and register MeerKAT
os.environ['PINT_CLOCK_OVERRIDE'] = str(clock_dir)
Observatory.clear_registry()
TopoObs(
    'meerkat',
    itrf_xyz=[5109360.133, 2006852.586, -3238948.127],
    tempo_code='m',
    itoa_code='MK',
    clock_file='mk2utc.clk',
    clock_fmt='tempo2',
    clock_dir=str(clock_dir),
    apply_gps2utc=True,
    overwrite=True,
)

# Load with PINT (BIPM2024) to get reference tdbld
model = get_model(par_file)
toas = get_TOAs(
    tim_file,
    model=model,
    usepickle=False,
    include_bipm=True,
    bipm_version='BIPM2024',
    planets=True,
)
pint_tdb = toas.table['tdbld'].value.astype(np.longdouble)

# Helpers for standalone path
def parse_mjd_string(mjd_str: str):
    if '.' in mjd_str:
        i, f = mjd_str.split('.')
        return np.longdouble(i), np.longdouble('0.' + f)
    return np.longdouble(mjd_str), np.longdouble(0.0)

def parse_clock_file(path: Path):
    mjds, offsets = [], []
    with open(path) as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            parts = line.split()
            if len(parts) < 2:
                continue
            try:
                mjds.append(np.longdouble(parts[0])); offsets.append(np.longdouble(parts[1]))
            except ValueError:
                pass
    return {'mjd': np.array(mjds, dtype=np.longdouble), 'offset': np.array(offsets, dtype=np.longdouble)}

def interp_clock(clock_data, mjd):
    mjds = clock_data['mjd']; offs = clock_data['offset']
    if len(mjds) == 0:
        return np.longdouble(0.0)
    if mjd <= mjds[0]:
        return offs[0]
    if mjd >= mjds[-1]:
        return offs[-1]
    idx = bisect_left(mjds, mjd)
    m0, m1 = mjds[idx-1], mjds[idx]
    o0, o1 = offs[idx-1], offs[idx]
    return o0 + (mjd - m0) * (o1 - o0) / (m1 - m0)

# Load clocks (mk2utc, gps2utc, bipm2024)
mk_clock = parse_clock_file(clock_dir / 'mk2utc.clk')
gps_clock = parse_clock_file(clock_dir / 'gps2utc.clk')
bipm_clock = parse_clock_file(clock_dir / 'tai2tt_bipm2024.clk')

# Parse TOAs with high precision
mjd_ints, mjd_fracs = [], []
with open(tim_file) as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith('#'):
            continue
        if line.startswith(('FORMAT','C ','JUMP','PHASE','MODE','INCLUDE')):
            continue
        parts = line.split()
        if len(parts) < 5:
            continue
        mi, mf = parse_mjd_string(parts[2])
        mjd_ints.append(mi); mjd_fracs.append(mf)

mjd_ints = np.array(mjd_ints, dtype=np.longdouble)
mjd_fracs = np.array(mjd_fracs, dtype=np.longdouble)

# Standalone TDB
tdb_custom = np.zeros(len(mjd_ints), dtype=np.longdouble)
for i, (mi, mf) in enumerate(zip(mjd_ints, mjd_fracs)):
    corr = (
        interp_clock(mk_clock, mi + mf)
        + interp_clock(gps_clock, mi + mf)
        + interp_clock(bipm_clock, mi + mf)
        - np.longdouble(32.184)
    )
    # Apply correction to fractional day and renormalize like PINT
    jd1, jd2 = mjds_to_jds_pulsar(mi, mf + corr / SECS_PER_DAY)
    t = Time(jd1, jd2, format='jd', scale='utc', location=mk_loc, precision=9)
    tdb_custom[i] = t.tdb.mjd

# Differences
corr_ns = (tdb_custom - pint_tdb) * SECS_PER_DAY * 1e9
print(f"Mean: {np.mean(corr_ns):.6f} ns, Std: {np.std(corr_ns):.6f} ns")
print(f"Min/Max: {np.min(corr_ns):.6f} / {np.max(corr_ns):.6f} ns")
print(f"Exact (<0.001 ns): {(np.abs(corr_ns) < 1e-3).sum()} / {len(corr_ns)}")
assert np.all(np.abs(corr_ns) < 0.001), "TDB mismatch > 0.001 ns"

plt.figure(figsize=(8,4))
plt.hist(corr_ns, bins=50, color='steelblue', edgecolor='black')
plt.xlabel('Custom TDB - PINT TDB (ns)')
plt.ylabel('Count')
plt.title('TDB Differences (ns)')
plt.grid(True, alpha=0.3)
plt.show()


2025-11-28 19:58:12.550 | DEBUG    | pint.toa:get_TOAs:195 - Using EPHEM = DE440 from the given model
2025-11-28 19:58:13.588 | DEBUG    | pint.toa:__init__:1377 - No pulse number flags found in the TOAs
2025-11-28 19:58:13.599 | DEBUG    | pint.toa:apply_clock_corrections:2232 - Applying clock corrections (include_bipm = True)
2025-11-28 19:58:13.786 | INFO     | pint.observatory:gps_correction:230 - Applying GPS to UTC clock correction (~few nanoseconds)
2025-11-28 19:58:13.787 | DEBUG    | pint.observatory:_load_gps_clock:108 - Loading global GPS clock file
2025-11-28 19:58:13.789 | DEBUG    | pint.observatory.clock_file:read_tempo2_clock_file:463 - Loading TEMPO2-format observatory clock correction file gps2utc.clk (/home/mattm/soft/JUG/data/clock/gps2utc.clk) with bogus_last_correction=False
2025-11-28 19:58:13.798 | DEBUG    | pint.observatory.clock_file:__init__:812 - Global clock file gps2utc.clk saving kwargs={'bogus_last_correction': False, 'valid_beyond_ends': False}
2025-11

Mean: 3.128264 ns, Std: 180.756732 ns
Min/Max: -314.014414 / 314.321369 ns
Exact (<0.001 ns): 3 / 10408


AssertionError: TDB mismatch > 0.001 ns